In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Data preprocessing useful function
from sklearn.preprocessing import LabelEncoder
# Function for spilting training & testing data set
from sklearn.model_selection import train_test_split
# ALgorithm
from sklearn import tree
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
# Functions for evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
from sktime.performance_metrics.forecasting import median_relative_absolute_error

In [2]:
df = pd.read_csv('train.csv')

In [3]:
important_num_cols = list(df.corr()["SalePrice"][(df.corr()["SalePrice"]>0.50) | (df.corr()["SalePrice"]<-0.50)].index)

df = df[important_num_cols]

In [4]:
new_df=df.dropna(axis=0, how='any').copy()
new_df.describe(include='all')

,OverallQual,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,GarageArea,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,6.099315,1971.267808,1984.865753,1057.429452,1162.626712,1515.463699,1.565068,6.517808,1.767123,472.980137,180921.195890
std,1.382997,30.202904,20.645407,438.705324,386.587738,525.480383,0.550916,1.625393,0.747315,213.804841,79442.502883
min,1.000000,1872.000000,1950.000000,0.000000,334.000000,334.000000,0.000000,2.000000,0.000000,0.000000,34900.000000
25%,5.000000,1954.000000,1967.000000,795.750000,882.000000,1129.500000,1.000000,5.000000,1.000000,334.500000,129975.000000
50%,6.000000,1973.000000,1994.000000,991.500000,1087.000000,1464.000000,2.000000,6.000000,2.000000,480.000000,163000.000000
75%,7.000000,2000.000000,2004.000000,1298.250000,1391.250000,1776.750000,2.000000,7.000000,2.000000,576.000000,214000.000000
max,10.000000,2010.000000,2010.000000,6110.000000,4692.000000,5642.000000,3.000000,14.000000,4.000000,1418.000000,755000.000000


In [5]:
x = df.drop(['SalePrice'],axis=1).copy()
y = df['SalePrice'].copy()

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
a = [2e-10,2e-6,2**8]
b = [2e-15,2e-10,2e-1]
c = [10,20,42]

mse_1_1 = 0
mse_1_2 = 0
mse_1_3 = 0
mse_2_1 = 0
mse_2_2 = 0
mse_2_3 = 0
mse_3_1 = 0
mse_3_2 = 0
mse_3_3 = 0

for k in c:
    x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.2, random_state = k)
    for i in a:
        for j in b:
            model=SVR(kernel='poly', C=i, epsilon=j)
            model.fit(x_validation,y_validation)
            
            validation_pred=model.predict(x_validation)
            mse=mean_squared_error(y_validation,validation_pred,squared=True)

            if i==2e-10 and j==2e-15 and k==10:
                mse_1_1 += mse
            if i==2e-10 and j==2e-10 and k==10:
                mse_1_2 += mse
            if i==2e-10 and j==2e-1 and k==10:
                mse_1_3 += mse
            if i==2e-6 and j==2e-15 and k==10:
                mse_2_1 += mse
            if i==2e-6 and j==2e-10 and k==10:
                mse_2_2 += mse
            if i==2e-6 and j==2e-1 and k==10:
                mse_2_3 += mse
            if i==2**8 and j==2e-15 and k==10:
                mse_3_1 += mse
            if i==2**8 and j==2e-10 and k==10:
                mse_3_2 += mse
            if i==2**8 and j==2e-1 and k==10:
                mse_3_3 += mse
                
print('MSE:')
print('mse_2_5:',round((mse_1_1),2))
print('mse_2_20:',round((mse_1_2),2))
print('mse_2_100:',round((mse_1_3),2))
print('mse_6_5:',round((mse_2_1),2))
print('mse_6_20:',round((mse_2_2),2))
print('mse_6_100:',round((mse_2_3),2))
print('mse_8_5:',round((mse_3_1),2))
print('mse_8_20:',round((mse_3_2),2))
print('mse_8_100:',round((mse_3_3),2))

MSE:
mse_2_5: 5671962693.45
mse_2_20: 5671962693.45
mse_2_100: 5671955353.98
mse_6_5: 5671962637.11
mse_6_20: 5671962637.11
mse_6_100: 5671955306.72
mse_8_5: 2475301988.12
mse_8_20: 2475301987.03
mse_8_100: 2475302248.97


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
a = [2e-10,2e-6,2**8]
b = [2e-15,2e-10,2e-1]
c = [10,20,42]

mse_1_1 = 0
mse_1_2 = 0
mse_1_3 = 0
mse_2_1 = 0
mse_2_2 = 0
mse_2_3 = 0
mse_3_1 = 0
mse_3_2 = 0
mse_3_3 = 0

for k in c:
    x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.2, random_state = k)
    for i in a:
        for j in b:
            model=SVR(kernel='poly', C=i, epsilon=j)
            model.fit(x_validation,y_validation)
            
            validation_pred=model.predict(x_validation)
            mse=mean_squared_error(y_validation,validation_pred,squared=True)

            if i==2e-10 and j==2e-15 and k==20:
                mse_1_1 += mse
            if i==2e-10 and j==2e-10 and k==20:
                mse_1_2 += mse
            if i==2e-10 and j==2e-1 and k==20:
                mse_1_3 += mse
            if i==2e-6 and j==2e-15 and k==20:
                mse_2_1 += mse
            if i==2e-6 and j==2e-10 and k==20:
                mse_2_2 += mse
            if i==2e-6 and j==2e-1 and k==20:
                mse_2_3 += mse
            if i==2**8 and j==2e-15 and k==20:
                mse_3_1 += mse
            if i==2**8 and j==2e-10 and k==20:
                mse_3_2 += mse
            if i==2**8 and j==2e-1 and k==20:
                mse_3_3 += mse
                
print('MSE:')
print('mse_2_5:',round((mse_1_1),2))
print('mse_2_20:',round((mse_1_2),2))
print('mse_2_100:',round((mse_1_3),2))
print('mse_6_5:',round((mse_2_1),2))
print('mse_6_20:',round((mse_2_2),2))
print('mse_6_100:',round((mse_2_3),2))
print('mse_8_5:',round((mse_3_1),2))
print('mse_8_20:',round((mse_3_2),2))
print('mse_8_100:',round((mse_3_3),2))

MSE:
mse_2_5: 5807918315.58
mse_2_20: 5807918315.58
mse_2_100: 5807927539.07
mse_6_5: 5807918282.39
mse_6_20: 5807918282.39
mse_6_100: 5807927494.28
mse_8_5: 2421539282.25
mse_8_20: 2421539282.25
mse_8_100: 2421543721.09


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
a = [2e-10,2e-6,2**8]
b = [2e-15,2e-10,2e-1]
c = [10,20,42]

mse_1_1 = 0
mse_1_2 = 0
mse_1_3 = 0
mse_2_1 = 0
mse_2_2 = 0
mse_2_3 = 0
mse_3_1 = 0
mse_3_2 = 0
mse_3_3 = 0

for k in c:
    x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.2, random_state = k)
    for i in a:
        for j in b:
            model=SVR(kernel='poly', C=i, epsilon=j)
            model.fit(x_validation,y_validation)
            
            validation_pred=model.predict(x_validation)
            mse=mean_squared_error(y_validation,validation_pred,squared=True)

            if i==2e-10 and j==2e-15 and k==42:
                mse_1_1 += mse
            if i==2e-10 and j==2e-10 and k==42:
                mse_1_2 += mse
            if i==2e-10 and j==2e-1 and k==42:
                mse_1_3 += mse
            if i==2e-6 and j==2e-15 and k==42:
                mse_2_1 += mse
            if i==2e-6 and j==2e-10 and k==42:
                mse_2_2 += mse
            if i==2e-6 and j==2e-1 and k==42:
                mse_2_3 += mse
            if i==2**8 and j==2e-15 and k==42:
                mse_3_1 += mse
            if i==2**8 and j==2e-10 and k==42:
                mse_3_2 += mse
            if i==2**8 and j==2e-1 and k==42:
                mse_3_3 += mse
                
print('MSE:')
print('mse_2_5:',round((mse_1_1),2))
print('mse_2_20:',round((mse_1_2),2))
print('mse_2_100:',round((mse_1_3),2))
print('mse_6_5:',round((mse_2_1),2))
print('mse_6_20:',round((mse_2_2),2))
print('mse_6_100:',round((mse_2_3),2))
print('mse_8_5:',round((mse_3_1),2))
print('mse_8_20:',round((mse_3_2),2))
print('mse_8_100:',round((mse_3_3),2))

MSE:
mse_2_5: 7804553914.58
mse_2_20: 7804553914.58
mse_2_100: 7804553914.58
mse_6_5: 7804553857.09
mse_6_20: 7804553857.09
mse_6_100: 7804553857.09
mse_8_5: 4977249018.52
mse_8_20: 4977249018.52
mse_8_100: 4977273941.47


In [9]:
#三次平均
mse_2_5 = [5671962693.45, 5807918315.58, 7804553914.58]
mse_2_20 = [5671962693.45, 5807918315.58, 7804553914.58]
mse_2_100 = [5671955353.98, 5807927539.07, 7804553914.58]
mse_6_5 = [5671962637.11, 5807918282.39, 7804553857.09]
mse_6_20 = [5671962637.11, 5807918282.39, 7804553857.09]
mse_6_100 = [5671955306.72, 5807927494.28, 7804553857.09]
mse_8_5 = [2475301988.12, 2421539282.25, 4977249018.52]
mse_8_20 = [2475301987.03, 2421539282.25, 4977249018.52]
mse_8_100 = [2475302248.97, 2421543721.09, 4977273941.47]

print('mse_2_5:',round(np.mean(mse_2_5),2),'+/-',round(np.std(mse_2_5),2))
print('mse_2_20:',round(np.mean(mse_2_20),2),'+/-',round(np.std(mse_2_20),2))
print('mse_2_100:',round(np.mean(mse_2_100),2),'+/-',round(np.std(mse_2_100),2))
print('mse_6_5:',round(np.mean(mse_6_5),2),'+/-',round(np.std(mse_6_5),2))
print('mse_6_20:',round(np.mean(mse_6_20),2),'+/-',round(np.std(mse_6_20),2))
print('mse_6_100:',round(np.mean(mse_6_100),2),'+/-',round(np.std(mse_6_100),2))
print('mse_8_5:',round(np.mean(mse_8_5),2),'+/-',round(np.std(mse_8_5),2))
print('mse_8_20:',round(np.mean(mse_8_20),2),'+/-',round(np.std(mse_8_20),2))
print('mse_8_100:',round(np.mean(mse_8_100),2),'+/-',round(np.std(mse_8_100),2))

mse_2_5: 6428144974.54 +/- 974849444.95
mse_2_20: 6428144974.54 +/- 974849444.95
mse_2_100: 6428145602.54 +/- 974849386.61
mse_6_5: 6428144925.53 +/- 974849439.5
mse_6_20: 6428144925.53 +/- 974849439.5
mse_6_100: 6428145552.7 +/- 974849381.27
mse_8_5: 3291363429.63 +/- 1192303169.29
mse_8_20: 3291363429.27 +/- 1192303169.54
mse_8_100: 3291373303.84 +/- 1192313777.16


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
model=SVR(kernel='poly', C=2**8, epsilon=2e-10)
model.fit(x_train,y_train)
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

y_pred_benchmark = train_pred*1.1
train_mse = mean_squared_error(y_train,train_pred,squared=True)
train_rmse = mean_squared_error(y_train,train_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train,train_pred)
train_smape = 2*np.mean(np.abs(train_pred-y_train)/(np.abs(train_pred)+np.abs(y_train)))*100
train_mae = mean_absolute_error(y_train, train_pred)
train_mdrae = median_relative_absolute_error(y_train, train_pred, y_pred_benchmark=y_pred_benchmark)

y_pred_benchmark = test_pred*1.1
test_mse = mean_squared_error(y_test,test_pred,squared=True)
test_rmse = mean_squared_error(y_test,test_pred, squared=False)
test_mape = mean_absolute_percentage_error(y_test,test_pred)
test_smape = 2*np.mean(np.abs(test_pred-y_test)/(np.abs(test_pred)+np.abs(y_test)))*100
test_mae = mean_absolute_error(y_test, test_pred)
test_mdrae = median_relative_absolute_error(y_test, test_pred, y_pred_benchmark=y_pred_benchmark)

print('train_mse : ', np.round(train_mse,2))
print('train_rmse: ', np.round(train_rmse,2))
print('train_mape: ', np.round(train_mape,2))
print('train_smape: ', np.round(train_smape,2))
print('train_mae: ', np.round(train_mae,2))
print('train_mdrae: ', np.round(train_mdrae,2))
print('')
print('test_mse : ', np.round(test_mse,2))
print('test_rmse: ', np.round(test_rmse,2))
print('test_mape: ', np.round(test_mape,2))
print('test_smape: ', np.round(test_smape,2))
print('test_mae: ', np.round(test_mae,2))
print('test_mdrae: ', np.round(test_mdrae,2))

train_mse :  2078142253.24
train_rmse:  45586.65
train_mape:  0.18
train_smape:  16.02
train_mae:  29439.45
train_mdrae:  0.74

test_mse :  13527477217.53
test_rmse:  116307.68
test_mape:  0.2
test_smape:  15.09
test_mae:  33617.17
test_mdrae:  0.68
